In [ ]:
import pandas as pd
import numpy as np
import altair as alt
from altair import datum
alt.data_transformers.disable_max_rows()

In [ ]:

#highlighting important NPIs in WA
data = {'date': [ "2020-03-23", "2020-06-01", "2020-11-18", "2021-02-14"], 'event':[ "Stay at home", "Stay at home lifted", "Closing restaurants", "Reopening restaurants"]}

npidf = pd.DataFrame(data)
npidf.date = pd.to_datetime(npidf.date)

rule = alt.Chart(npidf).mark_rule(
    color="black",
    strokeWidth=2, 
    opacity = 0.3
).encode(
    alt.X('date:T', axis=alt.Axis(title=None))
).properties(
    width=800,
    height=300
)

text = alt.Chart(npidf).mark_text(
    align='left',
    baseline='middle',
    dx=2,
    dy=-135,
    size=13
).encode(
    alt.X('date:T',axis=alt.Axis(title=None)),
    text='event',
    color=alt.value('#000000')
).properties(
    width=800,
    height=300
)

In [ ]:
north_percent_intro= "../data-files/north_percent_intro.csv"
south_percent_intro = "../data-files/south_percent_intro.csv"
percent_intro_mob = "../data-files/percent_intro_from_mob.csv"

In [ ]:

north_df = pd.read_csv(north_percent_intro)
south_df = pd.read_csv(south_percent_intro)
mob_df = pd.read_csv(percent_intro_mob)
north_rt_df = pd.read_csv("../data-files/combined_rt_north.csv")

south_rt_df = pd.read_csv("../data-files/combined_rt_south.csv")

In [ ]:
mob_df = mob_df[mob_df.date < "2022-03-02"]
mob_df = mob_df[mob_df.date > "2020-02-01"]

In [ ]:
mob_df = mob_df.drop(columns = ['Unnamed: 0'])
mob_df = mob_df.rename(columns={"Region": "Region (mobility data)"})

In [ ]:
mob_long = mob_df.melt('date', var_name='Region (mobility data)', value_name='movement')

In [ ]:
mob_long

In [ ]:
percent_cases_from_intro_mob = alt.Chart(mob_long, width = 750).mark_line(interpolate='monotone', opacity = 1.0, color = "black").encode(
    alt.X('date:T', axis=alt.Axis(title=None, grid=False)),
    alt.Y('movement:Q',axis=alt.Axis(title="", grid=False)), 
    alt.StrokeDash("Region (mobility data):N", legend=alt.Legend(orient = "none", legendX = 870, legendY= 50, labelFontSize = 12, titleFontSize = 12))).properties(
    width=800,
    height=400
)


percent_cases_from_intro_mob

In [ ]:
north_test = north_df[north_df.date < "2022-03-02"]
south_test = south_df[south_df.date <"2022-03-02"]

In [ ]:
north_test["Region (sequence data)"] = "North King County"
south_test["Region (sequence data)"] = "South King County"

combined_percent = pd.concat([ north_test, south_test], ignore_index=True)

In [ ]:
combined_percent

In [ ]:
line1 = alt.Chart(combined_percent).mark_area(interpolate='monotone').encode(
    alt.X('date:T', axis=alt.Axis(title="", grid=False,  format="%B %Y")),
    alt.Y('lower_hpd_log_50',axis=alt.Axis(title="Percent of cases due to introductions", grid=False, format='%')),
    alt.Y2('upper_hpd_log_50' ),
    alt.Color("Region (sequence data):N", legend=alt.Legend(offset = -140, labelFontSize = 12, titleFontSize = 12))
).properties(
    width=1000,
    height=300
).transform_filter(
    (datum.lower_hpd_log_50 >0) & (datum.upper_hpd_log_50 < 1)
)

band1 = alt.Chart(combined_percent).mark_area(
    opacity=0.3, interpolate='monotone'
).encode(
    alt.X('date:T', axis=alt.Axis(title="", grid=False)),
    alt.Y('lower_hpd_log_95', axis=alt.Axis(title="", grid=False)),
    alt.Y2('upper_hpd_log_95'), 
    alt.Color("Region (sequence data):N")
).properties(
    width=1000,
    height=300
).transform_filter(
    (datum.lower_hpd_log_95 >0) & (datum.upper_hpd_log_95 < 0.7)
)

band1 + line1

In [ ]:
percent_case_intro = band1+ line1
percent_case_intro.configure_axis(
    labelFontSize=14,
    titleFontSize=14
)

In [ ]:
stream_south = alt.Chart(south_rt_df, title = "South King County").mark_area(interpolate='monotone', opacity = .7 ,color = "#f58518", clip = True).encode(
    alt.X('date:T', axis=alt.Axis(title="", grid=False,format="%B %Y")),
    alt.Y('mean_percent',axis=alt.Axis(title="", grid=False), stack = False, scale=alt.Scale(domain=(0, 2.5))),
    #alt.Y2('upper_hpd_log_50' ), 
    alt.Color('Contribution:N', scale=alt.Scale(domain = ['Local', "Other King County Region", "Outside King County"], range = [ "#4c90c0", "#ceb541", "#df4327"]),
             legend=alt.Legend(offset = -160, labelFontSize = 12, titleFontSize = 12))
).properties(
    width=475,
    height=300
).transform_filter(
    (datum.mean_percent >0) & (datum.mean_percent < 2.5)
)

stream_north = alt.Chart(north_rt_df, title = "North King County").mark_area(interpolate='monotone', opacity = .7 ,color = "#f58518", clip = True).encode(
    alt.X('date:T', axis=alt.Axis(title="", grid=False,format="%B %Y")),
    alt.Y('mean_percent',axis=alt.Axis(title="Local Rt", grid=False),stack = False, scale=alt.Scale(domain=(0, 2.5))),
    #alt.Y2('upper_hpd_log_50' ), 
    alt.Color('Contribution:N', scale=alt.Scale(domain = ['Local', "Other King County Region", "Outside King County"], range = [ "#4c90c0", "#ceb541", "#df4327"]))
).properties(
    width=475,
    height=300
).transform_filter(
    (datum.mean_percent >0) & (datum.mean_percent < 2.5)
)

one_line = alt.Chart(pd.DataFrame({'y': [1.0]})).mark_rule(strokeDash=[1,1]).encode(y='y')

In [ ]:
rt_streamplot = ((stream_north +one_line) | (stream_south+one_line))
rt_streamplot

In [ ]:
combined_percent_intro = (percent_case_intro + percent_cases_from_intro_mob)
combined_percent_intro

In [ ]:
figure5 = (combined_percent_intro & rt_streamplot).resolve_scale(color = "independent").configure_axis(
    labelFontSize=14,
    titleFontSize=14
)
figure5

In [ ]:
figure5.save("../figures/figure5.html")

In [ ]:
figure5 = persist & percent_case_intro
figure5 = figure5.configure_axis(
    labelFontSize=14,
    titleFontSize=14
)
figure5

In [ ]:
#figure5.save("figure5_percent.png")